In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import json
from datetime import datetime

import requests
from bs4 import BeautifulSoup

In [29]:
caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options = Options()

options.add_experimental_option("detach", True)

options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

In [30]:
url = 'https://www.google.com/finance/quote/NVDA:NASDAQ?hl=ja&window=6M'
driver.get(url)

time.sleep(2)

logs = driver.get_log('performance')

In [31]:
batch_request_url = None
headers = {}
post_data = None

for entry in logs:
    log = json.loads(entry['message'])['message']
    try:
        if 'Network.requestWillBeSent' in log['method']:
            request_url = log['params']['request']['url']
            if 'https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=' in request_url:
                batch_request_url = request_url
                headers = log['params']['request']['headers']
                if 'postData' in log['params']['request']:
                    post_data = log['params']['request']['postData']
                break
    except KeyError:
        continue

driver.quit()

In [32]:
if batch_request_url:
    print(batch_request_url)
    if post_data:
        response = requests.post(batch_request_url, headers=headers, data=post_data)
    else:
        response = requests.get(batch_request_url, headers=headers)

    print("Status Code:", response.status_code)
    print("Response Text:", response.text)
    try:
        data = response.json()
        print(data)
    except json.JSONDecodeError:
        print("レスポンスのJSONデコードに失敗しました")
else:
    print("バッチリクエストURLが見つかりませんでした")

https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=AiCwsd&source-path=%2Ffinance%2Fquote%2FNVDA%3ANASDAQ&f.sid=1171123554744463663&bl=boq_finance-ui_20240610.00_p0&hl=ja&soc-app=162&soc-platform=1&soc-device=1&_reqid=54813&rt=c
Status Code: 200
Response Text: )]}'

12417
[["wrb.fr","AiCwsd","[[[[\"NVDA\",\"NASDAQ\"],\"/m/07zllzd\",\"USD\",[[[1],[[[2024,1,23,16,null,null,null,[-14400]],[59.87,0,0,2,2,2],29465445],[[2024,1,24,16,null,null,null,[-14400]],[61.36,1.490000000000002,0.02488725572072828,2,2,3],56027076],[[2024,1,25,16,null,null,null,[-14400]],[61.62,1.75,0.02922999832971438,2,2,3],48277684],[[2024,1,26,16,null,null,null,[-14400]],[61.03,1.1600000000000037,0.019375313178553594,2,2,3],39030859],[[2024,1,29,16,null,null,null,[-14400]],[62.47,2.6000000000000014,0.043427426089861394,2,2,3],34873298],[[2024,1,30,16,null,null,null,[-14400]],[62.77,2.9000000000000057,0.04843828294638393,2,2,3],41073530],[[2024,1,31,16,null,null,null,[-14400]],[61.53,1.6600000000

In [43]:
def remove_before_bracket_and_after_newline(s):
    # '['前の文字を削除
    bracket_index = s.find('[')
    if bracket_index != -1:
        s = s[bracket_index:]

    # 改行以降の文字を削除
    newline_index = s.find('\n')
    if newline_index != -1:
        s = s[:newline_index]

    return s

res_text = remove_before_bracket_and_after_newline(response.text)

print(res_text)

[["wrb.fr","AiCwsd","[[[[\"NVDA\",\"NASDAQ\"],\"/m/07zllzd\",\"USD\",[[[1],[[[2024,1,23,16,null,null,null,[-14400]],[59.87,0,0,2,2,2],29465445],[[2024,1,24,16,null,null,null,[-14400]],[61.36,1.490000000000002,0.02488725572072828,2,2,3],56027076],[[2024,1,25,16,null,null,null,[-14400]],[61.62,1.75,0.02922999832971438,2,2,3],48277684],[[2024,1,26,16,null,null,null,[-14400]],[61.03,1.1600000000000037,0.019375313178553594,2,2,3],39030859],[[2024,1,29,16,null,null,null,[-14400]],[62.47,2.6000000000000014,0.043427426089861394,2,2,3],34873298],[[2024,1,30,16,null,null,null,[-14400]],[62.77,2.9000000000000057,0.04843828294638393,2,2,3],41073530],[[2024,1,31,16,null,null,null,[-14400]],[61.53,1.6600000000000037,0.027726741272757705,2,2,3],45379487],[[2024,2,1,16,null,null,null,[-14400]],[63.03,3.1600000000000037,0.05278102555537003,2,2,3],36914579],[[2024,2,2,16,null,null,null,[-14400]],[66.16,6.289999999999999,0.10506096542508768,2,2,2],47657765],[[2024,2,5,16,null,null,null,[-14400]],[69.33,9

In [48]:
def string_to_list(s):
    try:
        # 文字列をJSON形式としてパースしてリストに変換
        result = json.loads(s)
        return result
    except (ValueError, json.JSONDecodeError) as e:
        # エラーメッセージを表示
        print(f"変換エラー: {e}")
        return None

result = string_to_list(res_text)
print(type(result))

<class 'list'>


In [45]:
import ast

def string_to_list(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        # 変換に失敗した場合、元の文字列を返すか、適切なエラーメッセージを返す
        return None

list_values = ast.literal_eval(res_text)

ValueError: malformed node or string on line 1: <ast.Name object at 0x00000272EF4D5C00>

In [36]:
import json
import re

def extract_date_close(response_text):
    """
    提供されたレスポンステキストから日付と終値を抽出します。
    """
    # 不要な部分を削除し、JSONデータを抽出
    json_str = re.search(r'\[\["wrb.fr".*?"generic"\]\]', response_text, re.DOTALL).group()

    # JSONデータの読み込み
    data = json.loads(json_str)

    # 日付と終値の抽出
    date_close_pairs = []
    nested_data = json.loads(data[0][2])  # 二重引用符で囲まれた部分を再度JSONとしてロード

    # データ構造を確認し、正しいインデックスを使用
    for entry in nested_data[0][3][0][1]:
        date = entry[0][:3]  # 日付部分を取得
        close_price = entry[1][0]  # 終値を取得
        date_close_pairs.append((date, close_price))

    return date_close_pairs

date_close_pairs = extract_date_close(response.text)
for date, close in date_close_pairs:
    print(f"Date: {date}, Close Price: {close}")

IndexError: list index out of range

In [39]:
import json
import re

def parse_stock_data(text):
    # 正規表現を使用して必要な部分を抽出
    pattern = re.compile(r'\[\[\[\["NVDA","NASDAQ"\],"\/m\/07zllzd","USD",(\[\[\[.*?\]\]\])')
    match = pattern.search(text)
    if not match:
        return []

    # 抽出した部分をJSONとして読み込む
    data = json.loads(match.group(1))
    stock_prices = []
    for entry in data:
        for item in entry:
            if len(item) > 1:
                date_info = item[0]
                close_info = item[1]
                if date_info and close_info:
                    date = f"{date_info[0]}-{date_info[1]:02}-{date_info[2]:02}"
                    close_price = close_info[0]
                    stock_prices.append({"date": date, "close": close_price})

    return stock_prices

# テキストファイルを読み込む
with open('./test.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# 株価データを解析
stock_data = parse_stock_data(text)

# JSON形式に変換
json_data = json.dumps(stock_data, indent=4)

# JSONデータをファイルに保存
output_file = './stock_data.json'
with open(output_file, 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)

output_file


'./stock_data.json'

In [ ]:
def string_to_list(s):
    try:
        # 文字列をJSON形式としてパースしてリストに変換
        result = json.loads(s)
        return result
    except (ValueError, json.JSONDecodeError) as e:
        # エラーメッセージを表示
        print(f"変換エラー: {e}")
        return None

